<a href="https://colab.research.google.com/github/GuyRobot/AINotesBook/blob/main/EHealthVNCrawlerX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
step = 40
BASE_URL = "http://diendansuckhoe24h.net/"

# url = "https://suckhoetoday.com/p/179-Phong-benh-mua-dong.html/"
# url = "https://suckhoetoday.com/p/97-BENH-VA-CHUA-BENH-CHO-TRE-CON.html/"
# url = "https://suckhoetoday.com/p/98-BENH-DICH-THEO-MUA.html/"
# url = "https://suckhoetoday.com/p/185-Phong-benh-mua-he.html/"
url = "http://diendansuckhoe24h.net/"
# url = "https://suckhoetoday.com/p/93-CHE-DO-DINH-DUONG.html/"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
init_url = url

In [17]:
soup.find_all("ol", {"id": "forums"})[0].find_all("ol", {"class": "nodeList"})[0].find_all("li", {"class": "node"})[0].find_all("h3", {"class": "nodeTitle"})[0].find_all("a")[0].get("href")

'forums/suc-khoe-tre-em/'

In [ ]:
[a.get("href") for a in x.find_all("a")[0] for x in soup.find_all("ol", {"class": "nodeList"})]

NameError: ignored

In [ ]:
links = [a.get("href") for a in soup.find_all("a", {"class": "title"})]
links